In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
import yaml

import cftime

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import intake

import regrid_tools
import config
import util

# Read FOSI simulation

Run details from README
```text
Monthly restarts for POP & CICE come from a modified* OMIP2 (JRA55-do forcing) spinup (6 cycles)
FOSI (forced ocean--sea-ice) simulation. Details of this simulation are as follows:
    - CASE: g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005
    - CASEROOT: /glade/work/klindsay/cesm22_cases/SMYLE/$CASE
    - SRCROOT: /glade/work/klindsay/cesm2_tags/cesm2.2.0/
    - forcing:  JRA55-do v1.4, 1958-2018 (==> 61-year cycle)
    - spinup: 6 cycles (==> simyears 0001-0366)
    - years used for SMYLE ICs:   0306 (1958) - 0366 (2018)

* Modifications from CMIP6-OMIP2 were made to improve sea-ice and ocean BGC fields:
    -Use of full 1958-2018 (61-year) forcing cycle during spinup
    -Use of strong under-ice restoring to model prognostic freezing temperature (TFZ)
    -Reduced deep isopycnal mixing (kappa_isop_deep = 0.1, instead of CESM2-default of 0.2)
    -Enhanced sea ice albedoes:
            r_snw = 1.6
            dt_mlt = 0.5
            rsnw_mlt = 1000.
```

In [3]:
year_range = 1990, 2018

In [4]:
src_grid = regrid_tools.grid("POP_gx1v7")
src_grid

exists: /glade/scratch/mclong/tmp/regridding/POP_gx1v7.nc


grid: POP_gx1v7
dims: (384, 320)
file: /glade/scratch/mclong/tmp/regridding/POP_gx1v7.nc

In [5]:
dst_grid = regrid_tools.grid("latlon", **config.config_dict["flux-dst-grid-kwargs"])
dst_grid

exists: /glade/scratch/mclong/tmp/regridding/latlon_1.0x1.0_lon0=-180.0.nc


grid: latlon_latlon_1.0x1.0_lon0=-180.0
dims: (180, 360)
file: /glade/scratch/mclong/tmp/regridding/latlon_1.0x1.0_lon0=-180.0.nc

In [6]:
regrid_obj = regrid_tools.regridder(src_grid, dst_grid, method="conserve", clobber=False)
regrid_obj

source grid dims: (384, 320)
destination grid dims: (180, 360)


regridder POP_gx1v7.nc --> latlon_1.0x1.0_lon0=-180.0.nc

In [7]:
dso_grid = util.generate_latlon_grid(**config.config_dict["flux-dst-grid-kwargs"])[["area"]]
dso_grid

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Data variables:
    area     (lat, lon) float64 1.079e+08 1.079e+08 ... 1.079e+08 1.079e+08

In [8]:
time_daily, time_daily_bnds = util.gen_daily_cftime_coord(year_range)
time_daily_num = cftime.date2num(time_daily, time_daily.encoding['units'])
time_daily

<xarray.DataArray 'time' (time: 10592)>
array([cftime.DatetimeGregorian(1990, 1, 1, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1990, 1, 2, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1990, 1, 3, 12, 0, 0, 0, has_year_zero=False),
       ...,
       cftime.DatetimeGregorian(2018, 12, 29, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2018, 12, 30, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2018, 12, 31, 12, 0, 0, 0, has_year_zero=False)],
      dtype=object)
Dimensions without coordinates: time
Attributes:
    bounds:   time_bnds

In [9]:
curator = util.curate_flux_products()
curator

{'description': 'Flux products for transport modeling', 'plugins': {'source': [{'module': 'intake_xarray'}]}, 'sources': {'fgapo.carboscope.apo99_v2020': {'args': {'urlpath': '/glade/work/mclong/sno-analysis/flux-products/fgapo_ocn.CarboScope.apo99_v2020.nc', 'xarray_kwargs': {'decode_times': False}}, 'description': 'APO fluxes from CarboScope inversion apo99_v2020', 'driver': 'netcdf'}, 'fgco2.MPI-SOM-FFN': {'args': {'urlpath': '/glade/work/mclong/sno-analysis/flux-products/fgco2.MPI-SOM-FFN.v2018.monclim_2009-2018.nc', 'xarray_kwargs': {'decode_times': False}}, 'description': 'An observation-based global monthly gridded sea surface pCO2 product from 1982 onward and its monthly climatology. Citation: Landschützer, P., Gruber, N., Bakker, D. C. E.: Decadal variations and trends of the global ocean carbon sink, Global Biogeochemical Cycles, 30, doi:10.1002/2015GB005359, 2016', 'driver': 'netcdf'}, 'fgco2.cesm_fosi_smyle': {'args': {'urlpath': '/glade/work/mclong/sno-analysis/flux-produc

In [10]:
cluster, client = util.get_ClusterClient()
cluster.scale(12)

client

/glade/work/mclong/miniconda3/envs/sno/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34679 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/34679/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/34679/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.40:36855,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/34679/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [ ]:
%%time
clobber = False

path = '/glade/campaign/cesm/development/espwg/SMYLE/initial_conditions/SMYLE-FOSI/ocn/proc/tseries'
case = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005'
year_offset = 1644

xr_open_kwargs = dict(chunks={'nlat': 16}, 
                      decode_coords=False, 
                      decode_times=False,
                     )

cesm_varnames = {
        'fgco2': {'varname': 'FG_CO2_2', 'freq': 'day_1', 'stream': 'pop.h.ecosys.nday1'},
        'fgo2': {'varname': 'STF_O2_2',  'freq': 'day_1', 'stream': 'pop.h.ecosys.nday1'},
        'fgn2': {'varname': ['SHF', 'SALT', 'TEMP'],  'freq': 'month_1', 'stream': 'pop.h'},    
}


nmolcm2s_molm2s = 1e-9 * 1e4

variables = ['fgco2', 'fgo2', 'fgn2']

dsets = {}
for v in variables:
    
    # output file
    file_out = f"{config.flux_product_dir}/{v}.{case}.{year_range[0]}0101-{year_range[1]}1231.nc"    
    if os.path.exists(file_out) and not clobber:
        print(f'exists: {file_out}')
        continue

    # get variable, stream and frequency
    varname = cesm_varnames[v]['varname']
    stream = cesm_varnames[v]['stream'] 
    freq = cesm_varnames[v]['freq']
    
    if freq == 'day_1':
        freq_str = 'daily'
        dateglob = '????????-????????'
    elif freq == 'month_1':
        freq_str = 'monthly'        
        dateglob = '??????-??????'
    

    # if varname is a list, assume it's a derived variable
    if isinstance(varname, list):
        ds_list = []
        for varname_i in varname:
            file = sorted(glob(f'{path}/{freq}/{case}.{stream}.{varname_i}.{dateglob}.nc'))
            assert len(file) == 1   
            ds_list.append(xr.open_dataset(file[0], **xr_open_kwargs))
        ds = xr.merge(ds_list)
        
        if v == 'fgn2':
            ds = ds.isel(z_t=0).rename({'SHF': 'hfds', 'SALT': 'sos', 'TEMP': 'tos'})
            ds = util.compute_fgn2(ds, scaleby=1.0).drop(['hfds', 'sos', 'tos'])
            ds.fgn2.attrs['units'] = 'mol/m^2/s'
        
        # append Jan from the last year of the simulation
        # so that the interpolation doesn't yeild NaNs
        if freq == 'month_1':
            assert 'days since' in ds.time.attrs['units']
            ds_jan_last = ds.isel(time=[-12])
            ds_jan_last['time'] = ds_jan_last.time + 365.
            ds_jan_last['time_bound'][0, :] = ds_jan_last.time_bound[0, :] + 365.
            ds = xr.concat((ds, ds_jan_last), dim='time')
    else:
        ds = xr.open_dataset(file[0], **xr_open_kwargs).rename({varname: v})
    
    time_bounds = ds[ds.time.bounds]

    # add year offset, change calendar to gregorian
    time_noleap = cftime.num2date(time_bounds.mean(axis=1), units=ds.time.units, calendar=ds.time.calendar)
    time_components = util.gen_time_components_variable(time_noleap, year_offset)
    
    time_gregorian_date = [
        cftime.DatetimeGregorian(*time_components[i, :]) 
        for i in range(time_components.shape[0])
    ]    
    time_gregorian_num = cftime.date2num(time_gregorian_date, units=time_daily.encoding['units'])
    ds['time'] = xr.DataArray(time_gregorian_num, dims=('time'))

    # interpolate to daily time (with leap days)
    ds = ds.interp(time=time_daily_num)
    ds['time'] = time_daily

    # subset variable, reverse sign convention
    ds = ds[[v]]    
    if v in ['fgco2', 'fgo2',]:
        with xr.set_options(keep_attrs=True):
            ds[v] = -1.0 * nmolcm2s_molm2s * ds[v]
        ds[v].attrs['units'] = 'mol/m^2/s'
    
    # regrid
    dso = regrid_obj(ds.compute())
    dso[v] = dso[v].rename({'nlat': 'lat', 'nlon': 'lon'})    
    for attr in ['coordinates', 'grid_loc', 'cell_methods']:
        if attr in dso[v].attrs:
            del dso[v].attrs[attr]
        
    dso["time"] = ds.time    
    dso[ds.time.bounds] = time_daily_bnds
    dso['lat'] = dso_grid['lat']
    dso['lon'] = dso_grid['lon']    
    dso['area'] = dso_grid['area']    

    # clean up attributes
    attrs_keys = list(dso.attrs.keys())
    for attr in attrs_keys:
        if attr not in ['model_doi_url']:
            del dso.attrs[attr]
    dso.attrs['source'] = case
    dso.attrs['note'] = 'Leap day values are linearly interpolated from adjacent data'

    # remove existing output
    if os.path.exists(file_out):
        print(f'removing: {file_out}')        
        os.remove(file_out)
    
    util.to_netcdf_clean(dso, file_out)
    
    curator.add_source(
        key=f"{v}.cesm_fosi_smyle",
        urlpath=file_out,
        description=f'CESM simulated flux from {case} at {freq_str} resolution',
    )

exists: /glade/work/mclong/sno-analysis/flux-products/fgco2.g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.19900101-20181231.nc
exists: /glade/work/mclong/sno-analysis/flux-products/fgo2.g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.19900101-20181231.nc


In [12]:
client.close()
cluster.close()

In [14]:
dso

<xarray.Dataset>
Dimensions:  (time: 10592, lat: 180, lon: 360)
Coordinates:
  * time     (time) object 1990-01-01 12:00:00 ... 2018-12-31 12:00:00
Dimensions without coordinates: lat, lon
Data variables:
    fgn2     (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    title:             g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2020-11-04 at 21:41:45.3
    cell_methods:      cell_methods = time: mean ==> the variable values are ...